In [ ]:
"""
Standalone AI Signal Analyzer
Rank signals, generate market analysis, and create trade recommendations using Gemini AI
"""

import json
from google import genai


class AISignalAnalyzer:
    """
    Standalone AI analyzer for trading signals and market analysis.
    Uses Google Gemini AI to rank signals and generate insights.
    """
    
    def __init__(self, gemini_api_key, model='gemini-2.0-flash-exp'):
        """
        Initialize AI analyzer
        
        Args:
            gemini_api_key: Your Gemini API key
            model: Gemini model to use (default: gemini-2.0-flash-exp)
        """
        self.api_key = gemini_api_key
        self.model = model
        self.client = genai.Client(api_key=gemini_api_key)
    
    def rank_signals(self, signals, symbol, current_price, batch_size=50):
        """
        Rank trading signals using AI scoring
        
        Args:
            signals: List of signal dicts with keys: signal, desc, strength, category, value
            symbol: Stock symbol
            current_price: Current stock price
            batch_size: Number of signals to process per API call (default: 50)
        
        Returns:
            List of signals with added fields: ai_score, ai_reasoning, rank
        """
        if len(signals) == 0:
            return signals
        
        print(f"🤖 Ranking {len(signals)} signals with AI...")
        
        # Process in batches to avoid token limits
        for i in range(0, len(signals), batch_size):
            batch = signals[i:i+batch_size]
            
            try:
                prompt = self._build_ranking_prompt(symbol, current_price, batch, i)
                response = self.client.models.generate_content(
                    model=self.model,
                    contents=prompt
                )
                
                self._parse_ranking_response(response.text, signals, i)
                
            except Exception as e:
                print(f"  ⚠️  Batch {i//batch_size + 1} error: {str(e)[:100]}")
                # Assign default scores on error
                for sig in batch:
                    if 'ai_score' not in sig:
                        sig['ai_score'] = 50
                        sig['ai_reasoning'] = 'Scoring error'
        
        # Sort by AI score
        signals.sort(key=lambda x: x.get('ai_score', 0), reverse=True)
        
        # Add rank numbers
        for rank, signal in enumerate(signals, 1):
            signal['rank'] = rank
        
        print(f"  ✅ Ranked {len(signals)} signals")
        return signals
    
    def _build_ranking_prompt(self, symbol, price, batch, start_idx):
        """Build prompt for signal ranking"""
        prompt = f"""Score these trading signals for {symbol} (Price: ${price:.2f}).
Score each signal 1-100 based on:
- Reliability: How trustworthy is this signal historically?
- Timing: Is this the right time to act on it?
- Risk/Reward: Does it offer good risk-adjusted returns?
- Actionability: How clear and specific is it?

SIGNALS:
"""
        for idx, sig in enumerate(batch, start_idx+1):
            prompt += f"{idx}. {sig['signal']}: {sig['desc']} [{sig['category']}]\n"
        
        prompt += """
Return ONLY valid JSON (no markdown, no backticks, no preamble):
{"scores":[{"n":1,"score":85,"why":"brief reason"},{"n":2,"score":72,"why":"reason"},...]}

Keep "why" under 40 characters. Be critical - use full 1-100 range."""
        
        return prompt
    
    def _parse_ranking_response(self, response_text, signals, start_idx):
        """Parse AI ranking response and update signals"""
        # Clean response
        response_text = response_text.strip()
        
        if '```json' in response_text:
            response_text = response_text.split('```json')[1].split('```')[0].strip()
        elif '```' in response_text:
            response_text = response_text.split('```')[1].split('```')[0].strip()
        
        # Extract JSON
        start = response_text.find('{')
        end = response_text.rfind('}')
        if start != -1 and end != -1:
            response_text = response_text[start:end+1]
        
        try:
            scores_data = json.loads(response_text)
            
            for score_item in scores_data.get('scores', []):
                sig_idx = score_item['n'] - 1
                if 0 <= sig_idx < len(signals):
                    signals[sig_idx]['ai_score'] = score_item.get('score', 50)
                    signals[sig_idx]['ai_reasoning'] = score_item.get('why', 'N/A')
        except json.JSONDecodeError as e:
            print(f"    ⚠️  JSON parse error: {str(e)[:50]}")
    
    def generate_market_analysis(self, symbol, market_data, signals, indicator_ratings):
        """
        Generate comprehensive market analysis
        
        Args:
            symbol: Stock symbol
            market_data: Dict with keys: price, change, rsi, macd, adx, volume_ratio
            signals: List of ranked signals (top 10 will be used)
            indicator_ratings: Dict with keys: trend_strength, momentum, volatility, volume
        
        Returns:
            String with detailed market analysis
        """
        print("🔍 Generating market analysis...")
        
        try:
            prompt = self._build_analysis_prompt(symbol, market_data, signals[:10], indicator_ratings)
            
            response = self.client.models.generate_content(
                model=self.model,
                contents=prompt
            )
            
            print("  ✅ Market analysis complete")
            return response.text
            
        except Exception as e:
            return f"Analysis error: {str(e)}"
    
    def _build_analysis_prompt(self, symbol, data, top_signals, ratings):
        """Build prompt for market analysis"""
        prompt = f"""Expert technical analysis for {symbol}.

CURRENT STATE:
- Price: ${data['price']:.2f} | Change: {data.get('change', 0):.2f}%
- RSI: {data.get('rsi', 50):.1f} | MACD: {data.get('macd', 0):.4f} | ADX: {data.get('adx', 0):.1f}
- Volume: {data.get('volume_ratio', 100):.0f}% of average

INDICATOR RATINGS (0-100):
- Trend: {ratings.get('trend_strength', 50)}/100
- Momentum: {ratings.get('momentum', 50)}/100
- Volatility: {ratings.get('volatility', 50)}/100
- Volume: {ratings.get('volume', 50)}/100

BIAS: {data.get('overall_bias', 'NEUTRAL')} ({data.get('bullish_count', 0)} bullish, {data.get('bearish_count', 0)} bearish signals)

TOP SIGNALS:
"""
        for i, sig in enumerate(top_signals, 1):
            prompt += f"{i}. {sig['signal']} [{sig.get('ai_score', 'N/A')}] - {sig['desc']}\n"
        
        prompt += """
Provide concise analysis:

1. MARKET CONTEXT: What's happening right now? (2-3 sentences)
2. KEY TAKEAWAY: Single most important insight for traders
3. TREND ANALYSIS: Short-term (1-5 days), Medium-term (1-4 weeks), Long-term (1-3 months) outlook
4. RISK ASSESSMENT: Main risks to watch for
5. TIMEFRAME: Best trading timeframe (day trading/swing/position)
6. CONFIDENCE: High/Medium/Low and why

Be specific, actionable, and concise. Avoid generic statements."""
        
        return prompt
    
    def generate_trade_recommendations(self, symbol, market_data, analysis_summary):
        """
        Generate specific trade recommendations with options strategies
        
        Args:
            symbol: Stock symbol
            market_data: Dict with keys: price, atr, volatility, overall_bias, trend_strength
            analysis_summary: Dict with overall bias and trend information
        
        Returns:
            Dict with trade recommendations (JSON format)
        """
        print("💰 Generating trade recommendations...")
        
        try:
            prompt = self._build_trade_prompt(symbol, market_data, analysis_summary)
            
            response = self.client.models.generate_content(
                model=self.model,
                contents=prompt
            )
            
            # Parse JSON response
            result = self._parse_trade_response(response.text)
            print("  ✅ Trade recommendations complete")
            return result
            
        except Exception as e:
            return {'error': str(e)}
    
    def _build_trade_prompt(self, symbol, data, summary):
        """Build prompt for trade recommendations"""
        current_price = data['price']
        atr = data.get('atr', current_price * 0.02)
        
        # Calculate suggested strikes
        put_spread_short = round(current_price - (2 * atr), 0)
        put_spread_long = put_spread_short - 5
        call_spread_short = round(current_price + (2 * atr), 0)
        call_spread_long = call_spread_short + 5
        
        prompt = f"""Generate specific trade recommendations for {symbol}.

MARKET DATA:
- Price: ${current_price:.2f}
- ATR (volatility): ${atr:.2f}
- Implied Volatility: {data.get('volatility', 30):.0f}%
- Overall Bias: {summary.get('overall_bias', 'NEUTRAL')}
- Trend Strength: {summary.get('trend_strength', 50)}/100

SUGGESTED OPTION SPREADS (5-wide, 30+ DTE):
Put Credit Spread: Sell ${put_spread_short:.0f} / Buy ${put_spread_long:.0f}
Call Credit Spread: Sell ${call_spread_short:.0f} / Buy ${call_spread_long:.0f}

Based on the technical analysis, provide:
1. PRIMARY STRATEGY: Best trade setup
2. ENTRY CRITERIA: Specific conditions to enter
3. RISK MANAGEMENT: Stop loss, position sizing
4. WIN PROBABILITY: Estimate based on current technicals
5. ALTERNATIVE: Second-best strategy

Return ONLY valid JSON (no markdown):
{{
  "primary_strategy": {{
    "type": "put_credit_spread",
    "short_strike": {put_spread_short},
    "long_strike": {put_spread_long},
    "rationale": "why this trade makes sense",
    "entry_criteria": "when to enter (be specific)",
    "max_loss": "$X per spread",
    "target_profit": "$X per spread",
    "win_probability": "X%",
    "stop_loss": "exit if..."
  }},
  "alternative_strategy": {{
    "type": "call_credit_spread",
    "short_strike": {call_spread_short},
    "long_strike": {call_spread_long},
    "rationale": "backup plan"
  }},
  "risk_notes": "key risks to monitor",
  "timeframe": "hold duration"
}}"""
        
        return prompt
    
    def _parse_trade_response(self, response_text):
        """Parse trade recommendation response"""
        response_text = response_text.strip()
        
        # Remove markdown formatting
        if '```json' in response_text:
            response_text = response_text.split('```json')[1].split('```')[0].strip()
        elif '```' in response_text:
            response_text = response_text.split('```')[1].split('```')[0].strip()
        
        # Extract JSON
        start = response_text.find('{')
        end = response_text.rfind('}')
        if start != -1 and end != -1:
            response_text = response_text[start:end+1]
        
        try:
            return json.loads(response_text)
        except json.JSONDecodeError:
            # Return as raw text if JSON parsing fails
            return {'raw_text': response_text}
    
    def analyze_complete(self, symbol, signals, market_data, indicator_ratings):
        """
        Complete AI analysis pipeline: rank signals, analyze market, recommend trades
        
        Args:
            symbol: Stock symbol
            signals: List of signal dicts
            market_data: Dict with current market state
            indicator_ratings: Dict with indicator scores
        
        Returns:
            Dict with all AI analysis results
        """
        print(f"\n🤖 Starting complete AI analysis for {symbol}...")
        
        # Stage 1: Rank signals
        ranked_signals = self.rank_signals(signals, symbol, market_data['price'])
        
        # Stage 2: Generate market analysis
        market_analysis = self.generate_market_analysis(
            symbol, 
            market_data, 
            ranked_signals, 
            indicator_ratings
        )
        
        # Stage 3: Generate trade recommendations
        trade_recs = self.generate_trade_recommendations(
            symbol,
            market_data,
            {
                'overall_bias': market_data.get('overall_bias', 'NEUTRAL'),
                'trend_strength': indicator_ratings.get('trend_strength', 50)
            }
        )
        
        return {
            'ranked_signals': ranked_signals[:20],  # Top 20
            'market_analysis': market_analysis,
            'trade_recommendations': trade_recs
        }


# ============ EXAMPLE USAGE ============

if __name__ == "__main__":
    """
    Example: How to use the AI Signal Analyzer
    """
    
    # Your API key
    API_KEY = "your-gemini-api-key-here"
    
    # Example signals from your technical analysis
    signals = [
        {
            'signal': 'GOLDEN CROSS',
            'desc': '50 MA crossed above 200 MA',
            'strength': 'STRONG BULLISH',
            'category': 'MA_CROSS',
            'value': 150.25
        },
        {
            'signal': 'RSI OVERSOLD',
            'desc': 'RSI(14): 28.5',
            'strength': 'BULLISH',
            'category': 'RSI',
            'value': 28.5
        },
        {
            'signal': 'MACD BULL CROSS',
            'desc': 'MACD crossed above signal',
            'strength': 'STRONG BULLISH',
            'category': 'MACD',
            'value': 0.45
        },
        {
            'signal': 'VOLUME SPIKE 2X',
            'desc': 'Volume: 2.3x average',
            'strength': 'SIGNIFICANT',
            'category': 'VOLUME',
            'value': 2.3
        }
    ]
    
    # Current market data
    market_data = {
        'price': 152.30,
        'change': 2.5,
        'rsi': 28.5,
        'macd': 0.45,
        'adx': 32.0,
        'volume_ratio': 230,
        'overall_bias': 'BULLISH',
        'bullish_count': 15,
        'bearish_count': 5,
        'atr': 3.50,
        'volatility': 35
    }
    
    # Indicator ratings
    indicator_ratings = {
        'trend_strength': 75,
        'momentum': 65,
        'volatility': 60,
        'volume': 80
    }
    
    # Initialize AI analyzer
    analyzer = AISignalAnalyzer(gemini_api_key=API_KEY)
    
    # Example 1: Rank signals only
    print("\n=== EXAMPLE 1: Rank Signals ===")
    ranked = analyzer.rank_signals(signals, symbol="AAPL", current_price=152.30)
    for sig in ranked[:5]:
        print(f"#{sig['rank']} [{sig['ai_score']}] {sig['signal']}: {sig['ai_reasoning']}")
    
    # Example 2: Generate market analysis
    print("\n=== EXAMPLE 2: Market Analysis ===")
    analysis = analyzer.generate_market_analysis(
        symbol="AAPL",
        market_data=market_data,
        signals=ranked,
        indicator_ratings=indicator_ratings
    )
    print(analysis[:500])  # Print first 500 chars
    
    # Example 3: Get trade recommendations
    print("\n=== EXAMPLE 3: Trade Recommendations ===")
    trades = analyzer.generate_trade_recommendations(
        symbol="AAPL",
        market_data=market_data,
        analysis_summary={
            'overall_bias': 'BULLISH',
            'trend_strength': 75
        }
    )
    print(json.dumps(trades, indent=2))
    
    # Example 4: Complete analysis pipeline
    print("\n=== EXAMPLE 4: Complete Analysis ===")
    complete = analyzer.analyze_complete(
        symbol="AAPL",
        signals=signals,
        market_data=market_data,
        indicator_ratings=indicator_ratings
    )
    
    print(f"\nTop Signal: {complete['ranked_signals'][0]['signal']}")
    print(f"AI Score: {complete['ranked_signals'][0]['ai_score']}")
    print(f"\nRecommended Trade: {complete['trade_recommendations'].get('primary_strategy', {}).get('type', 'N/A')}")


"""
INTEGRATION GUIDE:
==================

1. Install dependencies:
   pip install google-generativeai

2. Get your API key:
   https://aistudio.google.com/app/apikey

3. Basic usage:
   
   from ai_signal_analyzer import AISignalAnalyzer
   
   analyzer = AISignalAnalyzer(gemini_api_key="your-key")
   
   # Rank your signals
   ranked_signals = analyzer.rank_signals(
       signals=your_signals,
       symbol="AAPL",
       current_price=150.00
   )
   
   # Get market analysis
   analysis = analyzer.generate_market_analysis(
       symbol="AAPL",
       market_data=your_market_data,
       signals=ranked_signals,
       indicator_ratings=your_ratings
   )
   
   # Get trade recommendations
   trades = analyzer.generate_trade_recommendations(
       symbol="AAPL",
       market_data=your_market_data,
       analysis_summary={'overall_bias': 'BULLISH', 'trend_strength': 75}
   )

4. The analyzer expects specific data formats:
   
   Signals: List of dicts with keys: signal, desc, strength, category, value
   Market Data: Dict with keys: price, change, rsi, macd, adx, volume_ratio, etc.
   Indicator Ratings: Dict with keys: trend_strength, momentum, volatility, volume

5. Error handling:
   - All methods have try-except blocks
   - Failed AI calls return default values or error messages
   - No crashes - graceful degradation
"""